In [1]:
import pandas as pd

In [ ]:
datos = pd.read_csv("train.csv", sep=",")
# Display de dataset
train = pd.DataFrame(datos)
train.head()

## Manejo de datos faltantes




In [ ]:
missing_values = train.isnull().sum()
print("Valores faltantes por columna:")
print(missing_values[missing_values > 0])

In [ ]:
# Impute missing values
for column in train.columns:
    if train[column].isnull().any():
        if train[column].dtype == 'object':
            # Impute categorical columns with mode
            mode_value = train[column].mode()[0]
            train[column] = train[column].fillna(mode_value)
        elif train[column].dtype in ['int64', 'float64']:
            # Impute numerical columns with median
            median_value = train[column].median()
            train[column] = train[column].fillna(median_value)

# Verify no remaining missing values
print("\nMissing values after imputation:")
print(train.isnull().sum())

In [ ]:

from sklearn.preprocessing import LabelEncoder


# 1. Separar las características (X) de la variable objetivo (y)
# X contendrá todas las columnas EXCEPTO la que queremos predecir.
X = train.drop('RENDIMIENTO_GLOBAL', axis=1)

# y contendrá SOLAMENTE la columna que queremos predecir.
y_texto = train['RENDIMIENTO_GLOBAL']

# 2. Instanciar el LabelEncoder
le = LabelEncoder()

# 3. Aplicar fit_transform a 'y'
# Esto aprende las categorías y las transforma en números
y = le.fit_transform(y_texto)


print("Mapeo de LabelEncoder:")
# Itera sobre las clases aprendidas y muestra su número
for i, clase in enumerate(le.classes_):
    print(f"{clase}  ->  {i}")

print("\nPrimeras 5 etiquetas en texto:")
print(y_texto.head())

print("\nPrimeras 5 etiquetas codificadas (como números):")
print(y[:5])

print(f"\nDimensiones de X (features): {X.shape}")
print(f"Dimensiones de y (target): {y.shape}")

## Convertir variables categoricas one-hot




In [ ]:
# Identificar las columnas categóricas excluyendo RENDIMIENTO_GLOBAL
categorical_cols = X.select_dtypes(include=['object']).columns
#aplicar get_dummies a las features
X_transformado = pd.get_dummies(X, columns=categorical_cols)

print(f"Dimensiones de X después de One-Hot Encoding: {X_transformado.shape}")

In [ ]:
print(X_transformado.shape)

## Normalización


In [ ]:
numerical_cols = X_transformado.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_cols.remove('ID')
numerical_cols.remove('PERIODO_ACADEMICO')

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_transformado[numerical_cols] = scaler.fit_transform(X_transformado[numerical_cols])

display(X_transformado.head())

In [ ]:
display(X_transformado.head())
display(X_transformado.info())
display(X_transformado.describe())

In [ ]:

# 1. Guardar X_transformado como Parquet
print("Guardando X_transformado en formato Parquet...")
X_transformado.to_parquet('X_train_procesado.parquet', index=False)
print("¡Listo!")

# 2. Guardar 'y'
import numpy as np
print("Guardando y_train...")
np.save('y_train_procesado.npy', y)
print("¡Listo!")


# MODELO

In [21]:
X_train_procesado = pd.read_parquet('X_train_procesado.parquet')
print("X_train_procesado cargado.")

X_train_procesado cargado.


In [22]:
import numpy as np
y_train_procesado = np.load('y_train_procesado.npy')
print("y_train_procesado cargado.")

y_train_procesado cargado.


In [23]:
# Celda añadida: verificación y alineamiento seguro de X_train_procesado y y_train_procesado
import numpy as np
import pandas as pd


# Asegurar que ambas variables existen antes de proceder
if 'X_train_procesado' in globals() and 'y_train_procesado' in globals():
    len_X = X_train_procesado.shape[0]
    try:
        len_y = y_train_procesado.shape[0]
    except Exception:
        len_y = len(y_train_procesado)

    if len_X != len_y:
        print(f'Inconsistencia detectada: len(X)={len_X}, len(y)={len_y}. Alineando de forma segura...')
        # Preferir eliminar entradas con y NaN si y es Series
        if isinstance(y_train_procesado, (pd.Series, pd.DataFrame)):
            mask = ~y_train_procesado.isnull()
            y_train_procesado = y_train_procesado.loc[mask].reset_index(drop=True)
            # Si X tiene al menos tantas filas como mask.sum(), aplicamos la misma máscara por posición
            if X_train_procesado.shape[0] >= mask.sum():
                X_train_procesado = X_train_procesado.iloc[:mask.sum()].reset_index(drop=True)
            else:
                X_train_procesado = X_train_procesado.iloc[:y_train_procesado.shape[0]].reset_index(drop=True)
        else:
            # y es numpy array: truncar el más largo para alinear por posición
            if len_X > len_y:
                X_train_procesado = X_train_procesado.iloc[:len_y].reset_index(drop=True)
            else:
                y_train_procesado = y_train_procesado[:len_X]
        print('Alineamiento completado. Nuevas longitudes:', X_train_procesado.shape[0], getattr(y_train_procesado, 'shape', len(y_train_procesado)))
    else:
        print('X e y ya estaban alineados. len=', len_X)
else:
    print('Aviso: X_train_procesado o y_train_procesado no están definidas en el kernel. Ejecuta las celdas de preprocesamiento apropiadas antes de entrenar.')

X e y ya estaban alineados. len= 692500


In [24]:
test_df = pd.read_csv('test.csv', sep=',')
print("test_df cargado.")

test_df cargado.


In [25]:
missing_values_test = test_df.isnull().sum()
print("Valores faltantes por columna en test_df:")
print(missing_values_test[missing_values_test > 0])

Valores faltantes por columna en test_df:
E_VALORMATRICULAUNIVERSIDAD     2723
E_HORASSEMANATRABAJA           13379
F_ESTRATOVIVIENDA              13795
F_TIENEINTERNET                11539
F_EDUCACIONPADRE                9993
F_TIENELAVADORA                17259
F_TIENEAUTOMOVIL               18918
E_PAGOMATRICULAPROPIO           2807
F_TIENECOMPUTADOR              16439
F_TIENEINTERNET.1              11539
F_EDUCACIONMADRE               10223
dtype: int64


In [26]:
for column in test_df.columns:
    if test_df[column].isnull().any():
        if test_df[column].dtype == 'object':
            mode_value = test_df[column].mode()[0]
            test_df[column] = test_df[column].fillna(mode_value)
        elif test_df[column].dtype in ['int64', 'float64']:
            median_value = test_df[column].median()
            test_df[column] = test_df[column].fillna(median_value)

print("Missing values in test_df after imputation:")
print(test_df.isnull().sum()[test_df.isnull().sum() > 0])

Missing values in test_df after imputation:
Series([], dtype: int64)


In [27]:
categorical_cols_test = test_df.select_dtypes(include=['object']).columns
X_test_transformed = pd.get_dummies(test_df, columns=categorical_cols_test)

print(f"Dimensiones de X_test_transformed después de One-Hot Encoding: {X_test_transformed.shape}")

Dimensiones de X_test_transformed después de One-Hot Encoding: (296786, 1014)


In [28]:
train_cols = X_train_procesado.columns
test_cols = X_test_transformed.columns

missing_in_test = set(train_cols) - set(test_cols)
for c in missing_in_test:
    X_test_transformed[c] = 0

missing_in_train = set(test_cols) - set(train_cols)
X_test_transformed = X_test_transformed.drop(columns=list(missing_in_train))

X_test_transformed = X_test_transformed[train_cols]

print(f"Dimensiones de X_test_transformed después de la alineación de columnas: {X_test_transformed.shape}")
print(f"Columnas de X_test_transformed son iguales a las de X_train_procesado: {all(X_test_transformed.columns == X_train_procesado.columns)}")

/var/folders/4p/5cp_zzdd5h99gz5mkdx4h56h0000gn/T/ipykernel_35995/875242137.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_transformed[c] = 0
/var/folders/4p/5cp_zzdd5h99gz5mkdx4h56h0000gn/T/ipykernel_35995/875242137.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_transformed[c] = 0
/var/folders/4p/5cp_zzdd5h99gz5mkdx4h56h0000gn/T/ipykernel_35995/875242137.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  

Dimensiones de X_test_transformed después de la alineación de columnas: (296786, 1039)
Columnas de X_test_transformed son iguales a las de X_train_procesado: True


In [29]:
from sklearn.model_selection import train_test_split

print("Librería train_test_split importada.")

Librería train_test_split importada.


In [30]:
X_train_sampled_eval, X_val_sampled_eval, y_train_sampled_eval, y_val_sampled_eval = train_test_split(
    X_train_procesado, y_train_procesado, test_size=0.8, stratify=y_train_procesado, random_state=42
)

print(f"Dimensiones de X_train_sampled_eval: {X_train_sampled_eval.shape}")
print(f"Dimensiones de y_train_sampled_eval: {y_train_sampled_eval.shape}")
print(f"Dimensiones de X_val_sampled_eval: {X_val_sampled_eval.shape}")
print(f"Dimensiones de y_val_sampled_eval: {y_val_sampled_eval.shape}")

Dimensiones de X_train_sampled_eval: (138500, 1039)
Dimensiones de y_train_sampled_eval: (138500,)
Dimensiones de X_val_sampled_eval: (554000, 1039)
Dimensiones de y_val_sampled_eval: (554000,)


In [35]:
import unicodedata
import re
import pandas as pd

def sanitize_and_deduplicate_cols(df):
    """
    Sanitiza nombres de columnas y maneja duplicados secuencialmente.
    """
    def sanitize(col):
        # 1. Normalizar a ASCII (elimina acentos y diéresis)
        new_col = unicodedata.normalize('NFKD', col).encode('ascii', 'ignore').decode('utf-8')
        
        # 2. Reemplazar caracteres especiales y espacios por guiones bajos
        new_col = re.sub(r'[^a-zA-Z0-9_]', '_', new_col)
        
        # 3. Eliminar guiones bajos duplicados o iniciales/finales
        new_col = re.sub(r'_+', '_', new_col).strip('_')
        
        # 4. LightGBM no le gustan los brackets: 
        new_col = new_col.replace('[', '_').replace(']', '_')
        
        return new_col

    sanitized_cols = [sanitize(col) for col in df.columns]
    
    # --- Manejo de Duplicados ---
    counts = {}
    final_cols = []
    for col in sanitized_cols:
        original_col = col
        if col in counts:
            counts[col] += 1
            # Añadir sufijo secuencial (e.g., _2, _3)
            col = f"{original_col}_{counts[original_col]}"
        else:
            counts[col] = 1
        final_cols.append(col)
        
    df.columns = final_cols
    return df

# --- Aplicar la limpieza a todos los conjuntos de datos de características ---

# 1. Aplicar a los datos de entrenamiento completo
X_train_procesado = sanitize_and_deduplicate_cols(X_train_procesado)

# 2. Aplicar a los conjuntos de tuning (evaluación rápida)
X_train_sampled_eval = sanitize_and_deduplicate_cols(X_train_sampled_eval)
X_val_sampled_eval = sanitize_and_deduplicate_cols(X_val_sampled_eval)

# 3. Aplicar al conjunto de prueba
X_test_transformed = sanitize_and_deduplicate_cols(X_test_transformed)

print("Nombres de columnas sanitizados y duplicados renombrados exitosamente.")
print(f"Número de columnas final en X_train: {X_train_procesado.shape[1]}")

Nombres de columnas sanitizados y duplicados renombrados exitosamente.
Número de columnas final en X_train: 1039


## Entrenar Modelo (RandomForestClassifier) con Datos Muestreados


Entrenar un modelo RandomForestClassifier utilizando el *conjunto de entrenamiento reducido* (X_train_sampled, y_train_sampled).


In [ ]:
X_train_sampled, _, y_train_sampled, __ = train_test_split(X_train_procesado, y_train_procesado, test_size=0.9, stratify=y_train_procesado, random_state=42)

print(f"Dimensiones de X_train_sampled: {X_train_sampled.shape}")
print(f"Dimensiones de y_train_sampled: {y_train_sampled.shape}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier

print("Entrenando RandomForestClassifier con datos muestreados...")
model_sampled = RandomForestClassifier(random_state=42)
model_sampled.fit(X_train_sampled, y_train_sampled)

print("¡Modelo entrenado exitosamente con datos muestreados!")

Iniciando RandomizedSearchCV para LightGBM (en datos muestreados)...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.031051 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2072
[LightGBM] [Info] Number of data points in the train set: 92334, number of used features: 464
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.042831 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Total Bins 2072
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Number of data points in the train set: 92334, number of used features: 464
[LightGBM] [Info] Start training from score -1.386294
[LightG

In [ ]:
print("Realizando predicciones en el conjunto de validación muestreado...")
y_pred_val = model_sampled.predict(X_val_sampled_eval)

print("¡Predicciones realizadas exitosamente en el conjunto de validación!")
print(f"Primeras 5 predicciones en validación: {y_pred_val[:5]}")

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

print("Calculando métricas de evaluación...")

accuracy = accuracy_score(y_val_sampled_eval, y_pred_val)
f1 = f1_score(y_val_sampled_eval, y_pred_val, average='weighted')

print(f"Precisión (Accuracy) en el conjunto de validación: {accuracy:.4f}")
print(f"F1-Score ponderado en el conjunto de validación: {f1:.4f}")

In [ ]:
print("Realizando predicciones en los datos de prueba...")
predictions_encoded = model_sampled.predict(X_test_transformed)

print("¡Predicciones realizadas exitosamente!")
print(f"Primeras 5 predicciones codificadas: {predictions_encoded[:5]}")

In [ ]:
print("Inverse transformando las predicciones a etiquetas originales...")
predictions_decoded = le.inverse_transform(predictions_encoded)

print("¡Predicciones decodificadas exitosamente!")
print(f"Primeras 5 predicciones decodificadas: {predictions_decoded[:5]}")

In [ ]:
print("Creando el archivo de envío (submission.csv)...")
submission_df = pd.DataFrame({'ID': test_df['ID'], 'RENDIMIENTO_GLOBAL': predictions_decoded})

submission_df.to_csv('submission(7).csv', index=False)

print("¡Archivo submission.csv creado exitosamente!")
print("Primeras 5 filas del archivo de envío:")
print(submission_df.head())

Iniciando Solución Final 99 con Tuning y Feature Engineering...


# Tunning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

print("Importando librerías para RandomizedSearchCV...")

# 2. Definir el espacio de búsqueda de hiperparámetros
param_dist = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, 30, None],
    'min_samples_leaf': [1, 2, 4]
}

print("Espacio de hiperparámetros definido.")

# 3. Instanciar un RandomForestClassifier
rf = RandomForestClassifier(random_state=42)

# 4. Instanciar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=10,  # Número de configuraciones de parámetros a probar
    cv=3,       # 3-fold cross-validation
    scoring='f1_weighted', # Usar f1_weighted como métrica de evaluación
    random_state=42,
    n_jobs=-1   # Usar todos los procesadores disponibles
)

print("RandomizedSearchCV instanciado. Iniciando búsqueda...")

# 5. Ajustar RandomizedSearchCV a los datos de entrenamiento muestreados
random_search.fit(X_train_sampled_eval, y_train_sampled_eval)

print("Búsqueda de hiperparámetros completada.")

# 6. Imprimir los mejores parámetros encontrados
print("Mejores parámetros encontrados:")
print(random_search.best_params_)

# 7. Imprimir el mejor F1-score alcanzado
print("Mejor F1-Score (ponderado) encontrado:")
print(random_search.best_score_)

In [ ]:
print("Obteniendo los mejores hiperparámetros...")
best_params = random_search.best_params_

print("Instanciando RandomForestClassifier con los mejores hiperparámetros...")
model_full_data = RandomForestClassifier(**best_params, random_state=42)

print("Entrenando el modelo con el conjunto de datos de entrenamiento completo...")
model_full_data.fit(X_train_procesado, y_train_procesado)

print("¡Modelo entrenado exitosamente con los mejores hiperparámetros en los datos completos!")

In [ ]:
print("Realizando predicciones en los datos de prueba con el modelo de datos completos...")
predictions_full_data_encoded = model_full_data.predict(X_test_transformed)

print("¡Predicciones realizadas exitosamente con el modelo de datos completos!")
print(f"Primeras 5 predicciones codificadas del modelo completo: {predictions_full_data_encoded[:5]}")

In [ ]:
print("Inverse transformando las predicciones a etiquetas originales...")
predictions_final_decoded = le.inverse_transform(predictions_full_data_encoded)

print("¡Predicciones decodificadas exitosamente!")
print(f"Primeras 5 predicciones decodificadas: {predictions_final_decoded[:5]}")


Entrenando el modelo LightGBM final con el conjunto de datos completo...
¡Modelo LightGBM final entrenado exitosamente!


In [ ]:
print("Creando el archivo de envío (submission.csv)...")
submission_df = pd.DataFrame({'ID': test_df['ID'], 'RENDIMIENTO_GLOBAL': predictions_final_decoded})

submission_df.to_csv('submission (6).csv', index=False)

print("¡Archivo submission.csv creado exitosamente!")
print("Primeras 5 filas del archivo de envío:")
print(submission_df.head())


Evaluando el modelo LightGBM en el conjunto de validación...

--- Resultados de la Validación (LightGBM + Class Weight) ---
Precisión (Accuracy) en validación: 0.44062
F1-Score ponderado en validación: 0.42995

Realizando predicciones finales y generando archivo de envío...

¡Archivo submission_lgbm_final.csv creado exitosamente!
Sube este archivo a Kaggle para tu intento final de alta potencia.
Primeras 5 filas:
       ID RENDIMIENTO_GLOBAL
0  550236               bajo
1   98545               bajo
2  499179               bajo
3  782980               bajo
4  785185               bajo
